### NOWCODER SQL EXTRA HARD QUESTIONS www.nowcoder.com

In [ ]:
### Q12
SELECT 
    d.dept_no,
    d.emp_no,
    s.salary AS maxSalary
FROM dept_emp d
JOIN salaries s USING (emp_no)
WHERE 
    (s.salary,d.dept_no) IN (
        SELECT MAX(s.salary), d.dept_no
        FROM salaries s 
        JOIN dept_emp d 
        USING (emp_no)
        GROUP BY d.dept_no
    )
ORDER BY d.dept_no

In [ ]:
### Q21
SELECT 
    e.emp_no,
    s2.salary - s1.salary AS growth
FROM employees e
JOIN salaries s1 ON e.hire_date = s1.from_date AND s1.emp_no = e.emp_no
JOIN salaries s2 ON s2.emp_no = e.emp_no AND s2.to_date = '9999-01-01'
ORDER BY growth

In [ ]:
### Q25
SELECT
    e.emp_no,
    m.emp_no AS manager_no,
    s1.salary AS emp_salary,
    s2.salary AS manager_salary
FROM dept_emp e
LEFT JOIN dept_manager m
ON e.dept_no = m.dept_no AND m.to_date = '9999-01-01'
LEFT JOIN salaries s1
ON e.emp_no = s1.emp_no AND s1.to_date = '9999-01-01'
LEFT JOIN salaries s2
ON m.emp_no = s2.emp_no AND s2.to_date = '9999-01-01'
WHERE e.emp_no NOT IN (SELECT emp_no FROM dept_manager) AND s1.salary > s2.salary

In [ ]:
### Q26
SELECT
    d.dept_no,
    d.dept_name,
    t.title,
    COUNT(t.title) AS count     
FROM departments d
RIGHT JOIN dept_emp e
USING (dept_no)
LEFT JOIN titles t
ON e.emp_no = t.emp_no AND e.to_date = t.to_date AND e.to_date = '9999-01-01'
GROUP BY 
    d.dept_no,
    d.dept_name,
    t.title
ORDER BY 
    d.dept_no

In [ ]:
### Q28
SELECT
    c.name,
    COUNT(c.name) AS 'number'
FROM film f
LEFT JOIN film_category fc
USING (film_id)
LEFT JOIN category c
USING (category_id)
WHERE 
    f.description LIKE '%robot%' AND
    fc.category_id IN (
        SELECT fcc.category_id
        FROM film_category fcc
        GROUP BY fcc.category_id
        HAVING COUNT(fcc.film_id) >= 5
    )
GROUP BY c.name

In [ ]:
### Q70 First Method
SELECT
    l.date,
    FORMAT(IFNULL((
        (SELECT (1.0 * count_id2)
        FROM (
            SELECT 
                l2.user_id,
                COUNT(l2.user_id) AS count_id2, 
                l2.date
            FROM login l2
            WHERE 
                l2.user_id IN (
                    SELECT user_id 
                    FROM (
                        SELECT l3.user_id
                        FROM login l3
                        WHERE 
                            l3.date = l.date AND
                            l3.user_id NOT IN 
                                (SELECT user_id 
                                 FROM login 
                                 WHERE date < l3.date)
                        ) first_day3
                )
                AND
                l2.date = DATE_ADD(l.date, INTERVAL 1 DAY)
            GROUP BY l2.date 
            ) second_day) 
        /
        (SELECT count_id1 
        FROM (
            SELECT 
                 l1.user_id,
                 COUNT(l1.user_id) AS count_id1, 
                 l1.date 
            FROM login l1
            WHERE 
                l1.date = l.date AND
                l1.user_id NOT IN 
                    (SELECT user_id 
                     FROM login 
                     WHERE date < l1.date)
            GROUP BY l1.date 
            ) first_day
        )
    ), 0), 3) AS p
FROM login l
GROUP BY l.date
ORDER BY l.date

In [ ]:
### Q70 Idea from others
SELECT
    date,
    FORMAT(
        IFNULL(
            (
                (SUM(
                        CASE
                        WHEN
                        (user_id, DATE_ADD(date, INTERVAL 1 DAY)) IN (SELECT user_id, date FROM login) AND
                        (user_id, date) IN (SELECT user_id, MIN(date) FROM login GROUP BY user_id)
                        THEN 1
                        ELSE 0
                        END)) 
                /    
                (SUM(
                        CASE 
                        WHEN (user_id, date) IN (SELECT user_id, MIN(date) FROM login GROUP BY user_id)
                        THEN 1
                        ELSE 0
                        END))
            ), 0
        ), 3
    ) AS p
FROM login
GROUP BY date
ORDER BY date

In [ ]:
### Q76
SELECT 
    id,
    job,
    score,
    t_rank
FROM
    (SELECT 
        id,
        job,
        score,
        CAST(ROW_NUMBER() OVER (PARTITION BY job ORDER BY score DESC) AS SIGNED) AS t_rank,
        CAST(ROW_NUMBER() OVER (PARTITION BY job ORDER BY score) AS SIGNED) AS rank_reverse
    FROM grade
    ) rank_temp
WHERE t_rank = rank_reverse OR ABS(t_rank - rank_reverse) = 1
ORDER BY id

In [ ]:
### Q81
SELECT
    o.user_id,
    (
    SELECT date
    FROM order_info
    WHERE
        product_name IN ('C++', 'Python', 'Java')
        AND user_id = o.user_id
        AND status = 'completed'
        AND date > '2025-10-15'
    ORDER BY date
    LIMIT 1
    ) AS first_buy_date,
    (
    SELECT date
    FROM order_info
    WHERE
        product_name IN ('C++', 'Python', 'Java')
        AND user_id = o.user_id
        AND status = 'completed'
        AND date > '2025-10-15'
    ORDER BY date
    LIMIT 1, 1
    ) AS second_buy_date,
    (
    SELECT COUNT(*) OVER (PARTITION BY user_id) 
    FROM order_info 
    WHERE 
        product_name IN ('C++', 'Python', 'Java')
        AND user_id = o.user_id
        AND status = 'completed'
        AND date > '2025-10-15'
    LIMIT 1
    ) AS cnt
FROM order_info o
WHERE (SELECT COUNT(*) OVER (PARTITION BY user_id) FROM order_info WHERE user_id = o.user_id AND date > '2025-10-15' LIMIT 1) > 1
GROUP BY o.user_id
HAVING cnt > 1
ORDER BY o.user_id

In [ ]:
# Q81 Idea from others
SELECT 
    bd.user_id,
    MIN(bd.date) AS first_buy_date,
    MAX(bd.date) AS second_buy_date,
    bd.cnt
FROM (
    SELECT 
        *,
        ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY date) AS row_num,
        COUNT(user_id) OVER (PARTITION BY user_id) AS cnt
    FROM order_info
    WHERE 
        date > '2025-10-15'
        AND status = 'completed'
        AND product_name IN ('C++', 'Python', 'Java')
) bd
WHERE bd.cnt > 1 AND bd.row_num < 3
GROUP BY bd.user_id
ORDER BY bd.user_id

In [ ]:
# Q86
SELECT
    firstyr.job,
    firstyr.first_year_mon,
    firstyr.first_year_cnt,
    firstyr.second_year_month,
    secondyr.first_year_cnt AS second_year_cnt

FROM (
    SELECT 
        job,
        date,
        CONCAT(YEAR(date), '-', DATE_FORMAT(date, '%m')) AS first_year_mon,
        SUM(num) AS first_year_cnt,
        CONCAT(YEAR(DATE_ADD(date, INTERVAL 1 YEAR)),'-', DATE_FORMAT(DATE_ADD(date, INTERVAL 1 YEAR), '%m')) AS second_year_month
    FROM resume_info 
    GROUP BY job, first_year_mon
    ) firstyr

LEFT JOIN (
    SELECT 
        job,
        date,
        CONCAT(YEAR(date), '-', DATE_FORMAT(date, '%m')) AS first_year_mon,
        SUM(num) AS first_year_cnt,
        CONCAT(YEAR(DATE_ADD(date, INTERVAL 1 YEAR)),'-', DATE_FORMAT(DATE_ADD(date, INTERVAL 1 YEAR), '%m')) AS second_year_month
    FROM resume_info 
    GROUP BY job, first_year_mon
    ) secondyr

ON secondyr.first_year_mon = firstyr.second_year_month AND secondyr.job = firstyr.job
WHERE YEAR(firstyr.date) = '2025'

ORDER BY MONTH(firstyr.date) DESC, job DESC

In [ ]:
# Q86 Cleaner
SELECT
    firstyr.job,
    firstyr.first_year_mon,
    firstyr.first_year_cnt,
    secondyr.second_year_month,
    secondyr.second_year_cnt
FROM (
    SELECT 
        job,
        DATE_FORMAT(date, '%Y-%m') AS first_year_mon,
        SUM(num) AS first_year_cnt
    FROM resume_info
    WHERE LEFT(date, 4) = '2025'
    GROUP BY job, first_year_mon
    ) firstyr
LEFT JOIN (
    SELECT 
        job,
        DATE_FORMAT(date, '%Y-%m') AS second_year_month,
        SUM(num) AS second_year_cnt
    FROM resume_info 
    WHERE LEFT(date, 4) = '2026'
    GROUP BY job, second_year_month
    ) secondyr
ON RIGHT(secondyr.second_year_month, 2) = RIGHT(firstyr.first_year_mon, 2) AND secondyr.job = firstyr.job
ORDER BY firstyr.first_year_mon DESC, job DESC

In [ ]:
# Q91
WITH sum_grade AS (
    SELECT 
        *,
        SUM(IF(g.type='add',g.grade_num,-g.grade_num)) AS grade_num1
    FROM grade_info g
    LEFT JOIN user u
    ON g.user_id = u.id
    GROUP BY user_id
)

SELECT 
    user_id AS id,
    name,
    grade_num1 AS grade_num
FROM sum_grade
WHERE grade_num1 = (SELECT MAX(grade_num1) FROM sum_grade)
ORDER BY user_id